## Get all archives using http://s2-public-api-prod.us-west-2.elasticbeanstalk.com/corpus/download/

Need to find a way to filter all these articles and keep only those which can be of interest to us i.e. those talking about species.

Idealy we need to tag each article of interest with:
- tag of the specie it talks about
- tag of the genus
- tag of the family

We can base our work on: 
- paper title
- paper abstract

Our assumption is that is the author used a particular species for his article, he will mention it either in the title or the abstract (strong assumption).

First I will study papers' abstract. 

In a given gz file, we will need to drop all rows with no abstract. 

#### Imports

In [1]:
import glob
import json
import pandas as pd
import gzip
import os
import io
from langdetect import detect
import scispacy                                                        
import spacy                                                        
import en_core_sci_lg
from spacy import displacy

In [2]:
from typing import List, Dict

### Get data

In [3]:
data_dir = "/Users/chloesekkat/Documents/batch8_ceebios/data_open_source"

In [4]:
def get_gz_files(data_dir: str) -> List[Dict]:
    """
    Get 
    """
    for file in os.listdir(data_dir):
        json_list = []
        if file.endswith('.gz'):
            gz = gzip.open(os.path.join(data_dir, file), 'rb')
            f = io.BufferedReader(gz)
            for line in f.readlines():
                json_list.append(json.loads(line))
            gz.close()
        return json_list

In [5]:
json_list = get_gz_files(data_dir)

In [6]:
data = pd.DataFrame(json_list)

In [7]:
data.shape

(32229, 21)

In [8]:
def missing_data(data: pd.DataFrame) -> pd.DataFrame:
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return(tt)

In [9]:
missing_data(data)

,Total,Percent
id,0,0.00000
title,0,0.00000
paperAbstract,0,0.00000
authors,0,0.00000
inCitations,0,0.00000
outCitations,0,0.00000
year,254,0.78811
s2Url,0,0.00000
sources,0,0.00000
pdfUrls,0,0.00000


In [10]:
data.head()

,id,title,paperAbstract,authors,inCitations,outCitations,year,s2Url,sources,pdfUrls,...,journalName,journalVolume,journalPages,doi,doiUrl,pmid,fieldsOfStudy,magId,s2PdfUrl,entities
0,5cf3fcad3ee67c45f1c3d98c2b4bc22f683bfca7,Quantum phase diagrams and time-of-flight pict...,By treating the hopping parameter as a perturb...,"[{'name': 'Jun Zhang', 'ids': ['49050562']}, ...",[7dfd349892b456de2b60a9e9a4dedfbaeedf767f],"[b029856e343340fc7c034f514dc15a2ad49f6c1a, 7c8...",2016.0,https://semanticscholar.org/paper/5cf3fcad3ee6...,[],[],...,Laser Physics,26,095501,10.1088/1054-660X/26/9/095501,https://doi.org/10.1088/1054-660X%2F26%2F9%2F0...,,[Physics],2461559103,,[]
1,9758f471a6789d32e9d684856c1b257b1fbb5546,IPCC (Intergovernmental Panel on Climate Chang...,The European Science Foundation (ESF) and the ...,"[{'name': 'Richard C. J. Somerville', 'ids': [...","[eca662dc341726551b1e74e9bc86d6f9aa37b15e, efd...",[],2008.0,https://semanticscholar.org/paper/9758f471a678...,[],[],...,,,,,,,[],2902693084,,[]
2,73236ad2ede98f3f3f6acbf83a463b4afe2dd1fb,Increasing CRISPR Efficiency and Measuring Its...,Genome editing of human cluster of differentia...,"[{'name': 'Jenny Shapiro', 'ids': ['36049549'...",[],"[d83ebd27f91b75b472da63636596668da6c72059, 56c...",2020.0,https://semanticscholar.org/paper/73236ad2ede9...,[Medline],[],...,Molecular Therapy. Methods & Clinical Development,17,1097 - 1107,10.1016/j.omtm.2020.04.027,https://doi.org/10.1016/j.omtm.2020.04.027,32478125,"[Biology, Medicine]",3022279551,,[]
3,3d9a672fef4b9fe99cd948787e340d68fbf1513b,Don’t You Be Telling Me How Tah Talk: Educatio...,,"[{'name': 'LaQuita N Gresham', 'ids': ['811828...",[],"[3285d0b0374acfeaeca887ba5e884d62393d5e40, 136...",2014.0,https://semanticscholar.org/paper/3d9a672fef4b...,[],[],...,,,,,,,[Medicine],49447698,,[]
4,251045388ce98e901ccc7d22ae754a224c72dc28,Therapeutic effect of taurine against aluminum...,The aim of the study was to demonstrate the th...,"[{'name': 'Springer-Verlag Italia', 'ids': ['...",[077ca4833b07ff8fb39501d915f43d48976c7eab],"[fb93f765e84bfcbdc722c07762e247a544bae269, 21b...",2014.0,https://semanticscholar.org/paper/251045388ce9...,[],[],...,,,,,,,[Medicine],2327692075,,[]


In [11]:
type(data.iloc[3]["paperAbstract"])

str

We need to remove the rows with empty string.

In [12]:
def remove_empty_abstract(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove rows where paper abstract is an empty string.
    """
    where = df["paperAbstract"].values != ""
    return df[where]

In [13]:
data = remove_empty_abstract(data)

In [14]:
to_keep = [
    "title",
    "paperAbstract",
    "fieldsOfStudy"
]

def keep_columns(df: pd.DataFrame, cols_to_keep: List[str]) -> pd.DataFrame:
    """
    Return dataframe with wanted columns.
    """
    return df[cols_to_keep]

In [15]:
data = keep_columns(data, to_keep)

In [16]:
data.head()

,title,paperAbstract,fieldsOfStudy
0,Quantum phase diagrams and time-of-flight pict...,By treating the hopping parameter as a perturb...,[Physics]
1,IPCC (Intergovernmental Panel on Climate Chang...,The European Science Foundation (ESF) and the ...,[]
2,Increasing CRISPR Efficiency and Measuring Its...,Genome editing of human cluster of differentia...,"[Biology, Medicine]"
4,Therapeutic effect of taurine against aluminum...,The aim of the study was to demonstrate the th...,[Medicine]
6,Mechanism of 'crowd' in evolutionary MAS for m...,This work introduces a new evolutionary approa...,[Computer Science]


In [17]:
def keep_english_abstracts(df: pd.DataFrame) -> pd.DataFrame:
    """
    Keep only papers with abstract in english.
    """
    keep_indexes = [] # get lits of indexes that we will keep
    df = df.reset_index(drop=True)
    for i, element in enumerate(df["paperAbstract"]):
        try:
            res = detect(element)
            if res == 'en':
                keep_indexes.append(i)
        except:
            print('Error with: ', element)
            print(type(element))
    return df[df.index.isin(keep_indexes)]

In [18]:
data = keep_english_abstracts(data)

Error with:  2
<class 'str'>
Error with:  ............................................................................................................................................... 3
<class 'str'>
Error with:  3.
<class 'str'>
Error with:  3
<class 'str'>
Error with:  473
<class 'str'>
Error with:  5.
<class 'str'>


In [19]:
data.shape

(15332, 3)

In [20]:
data.head()

,title,paperAbstract,fieldsOfStudy
0,Quantum phase diagrams and time-of-flight pict...,By treating the hopping parameter as a perturb...,[Physics]
1,IPCC (Intergovernmental Panel on Climate Chang...,The European Science Foundation (ESF) and the ...,[]
2,Increasing CRISPR Efficiency and Measuring Its...,Genome editing of human cluster of differentia...,"[Biology, Medicine]"
3,Therapeutic effect of taurine against aluminum...,The aim of the study was to demonstrate the th...,[Medicine]
4,Mechanism of 'crowd' in evolutionary MAS for m...,This work introduces a new evolutionary approa...,[Computer Science]
